In [10]:
import temporal_graph
import matplotlib.pyplot as plt
import temporal_graph_for_training


import importlib
import torch

import temporal_graph_for_training
importlib.reload(temporal_graph_for_training)

from temporal_graph_for_training import TemporalGraph, TemporalTruthModel


In [11]:
temporal_graph = temporal_graph_for_training.TemporalGraph()




creating time intervall ...
the time intervall of the data goes
from: 2022-02-08 16:22:00 
to: 2022-10-14 20:48:00
we have the following time intervalls[(Timestamp('2022-02-08 16:22:00'), Timestamp('2022-02-15 16:22:00')), (Timestamp('2022-02-15 16:22:00'), Timestamp('2022-02-22 16:22:00')), (Timestamp('2022-02-22 16:22:00'), Timestamp('2022-03-01 16:22:00')), (Timestamp('2022-03-01 16:22:00'), Timestamp('2022-03-08 16:22:00')), (Timestamp('2022-03-08 16:22:00'), Timestamp('2022-03-15 16:22:00')), (Timestamp('2022-03-15 16:22:00'), Timestamp('2022-03-22 16:22:00')), (Timestamp('2022-03-22 16:22:00'), Timestamp('2022-03-29 16:22:00')), (Timestamp('2022-03-29 16:22:00'), Timestamp('2022-04-05 16:22:00')), (Timestamp('2022-04-05 16:22:00'), Timestamp('2022-04-12 16:22:00')), (Timestamp('2022-04-12 16:22:00'), Timestamp('2022-04-19 16:22:00')), (Timestamp('2022-04-19 16:22:00'), Timestamp('2022-04-26 16:22:00')), (Timestamp('2022-04-26 16:22:00'), Timestamp('2022-05-03 16:22:00')), (Timest

In [12]:
# inspect raw author values
authors = temporal_graph.comment_df["author"].head(10).tolist()
print("Sample authors from CSV:", authors)
print("Type of first author:", type(authors[0]))

# inspect user node keys
keys = list(temporal_graph.user_nodes.keys())[:10]
print("Sample user node keys:", keys)
print("Type of key:", type(keys[0]))

for u in temporal_graph.user_nodes.values():
    if u.comments:
        print("FOUND COMMENTS")
        print(len(u.comments))
        break

Sample authors from CSV: [2247, 20054, 11225, 11225, 11225, 11225, 11225, 11225, 11225, 11225]
Type of first author: <class 'int'>
Sample user node keys: ['2247', '20054', '11225', '2335', '15010', '20030', '14695', '20058', '18270', '17712']
Type of key: <class 'str'>
FOUND COMMENTS
87


In [13]:
for u in temporal_graph.user_nodes.values():
    
    if u.comments:
        c = next(iter(u.comments.values()))
        break
else:
    raise RuntimeError("No users with comments found!")

In [14]:
print("===== SANITY CHECKS =====")
print("Number of users:", len(temporal_graph.user_nodes))
print("Number of time intervals:",
      len(temporal_graph.time_intervall.time_intervalls))

# count comments per interval (global)
interval_counts = [
    sum(len(u.comments_in_intervall[t])
        for u in temporal_graph.user_nodes.values())
    for t in range(len(temporal_graph.time_intervall.time_intervalls))
]

print("Comments per interval (first 10):", interval_counts[:10])
print("Total comments:", sum(interval_counts))
print("=========================\n")


===== SANITY CHECKS =====
Number of users: 2251
Number of time intervals: 36
Comments per interval (first 10): [433, 2394, 4558, 5140, 5657, 4444, 4016, 3663, 3238, 2720]
Total comments: 164705



In [ ]:
# ======================================================
# STABLE MEMORY-SAFE SWEEP + PER-CLASS F1
# ======================================================

import torch
import itertools
import pandas as pd
import gc

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)

COMMENT_HIDDEN = 128
USER_HIDDEN = 128

# ------------------------------------------------------
# 1) Infer feature dimension
# ------------------------------------------------------
any_user = next(
    u for u in temporal_graph.user_nodes.values()
    if len(u.comments) > 0
)
any_comment = next(iter(any_user.comments.values()))
COMMENT_DIM = any_comment.feature_vec.shape[0]
print("COMMENT_DIM =", COMMENT_DIM)

# ------------------------------------------------------
# Define Focal Loss
# ------------------------------------------------------

class FocalLoss(torch.nn.Module):
    def __init__(self, gamma=2.0):
        super().__init__()
        self.gamma = gamma
        self.bce = torch.nn.BCEWithLogitsLoss(reduction='none')

    def forward(self, logits, targets):
        bce_loss = self.bce(logits, targets)
        probs = torch.sigmoid(logits)
        pt = torch.where(targets == 1, probs, 1 - probs)
        focal_weight = (1 - pt) ** self.gamma
        return (focal_weight * bce_loss).mean()




# ------------------------------------------------------
# 2) Temporal split
# ------------------------------------------------------
T = len(temporal_graph.time_intervall.time_intervalls)

T_TRAIN = int(0.6 * T)
T_VAL   = int(0.8 * T)

train_ts = list(range(0, T_TRAIN))
val_ts   = list(range(T_TRAIN, T_VAL))

print(f"Train: 0 → {T_TRAIN-1}")
print(f"Val  : {T_VAL-1}")


# ------------------------------------------------------
# 3) Per-class metrics
# ------------------------------------------------------
def compute_metrics(logits, labels, threshold):

    probs = torch.sigmoid(logits)
    preds = (probs > threshold).float()

    tp1 = ((preds == 1) & (labels >= 0.5)).sum().item()
    fp1 = ((preds == 1) & (labels < 0.5)).sum().item()
    fn1 = ((preds == 0) & (labels >= 0.5)).sum().item()

    tp0 = ((preds == 0) & (labels < 0.5)).sum().item()
    fp0 = ((preds == 0) & (labels >= 0.5)).sum().item()
    fn0 = ((preds == 1) & (labels < 0.5)).sum().item()

    prec1 = tp1 / (tp1 + fp1 + 1e-8)
    rec1  = tp1 / (tp1 + fn1 + 1e-8)
    f1_1  = 2 * prec1 * rec1 / (prec1 + rec1 + 1e-8)

    prec0 = tp0 / (tp0 + fp0 + 1e-8)
    rec0  = tp0 / (tp0 + fn0 + 1e-8)
    f1_0  = 2 * prec0 * rec0 / (prec0 + rec0 + 1e-8)

    macro_f1 = (f1_0 + f1_1) / 2.0
    acc = (preds == (labels >= 0.5).float()).float().mean().item()

    return {
        "precision_class_1": prec1,
        "recall_class_1": rec1,
        "precision_class_0": prec0,
        "recall_class_0": rec0,
        "f1_class_1": f1_1,
        "f1_class_0": f1_0,
        "macro_f1": macro_f1,
        "accuracy": acc
    }


# ------------------------------------------------------
# 4) Sweep configuration
# ------------------------------------------------------
thresholds  = [0.3, 0.5]
use_past_y_opts   = [False]
use_graph_opts    = [True]
use_comments_opts = [True]

EPOCHS = 15
PATIENCE = 3
LR = 5e-4  # slightly lower for stability

results = []
best_macro_f1 = -1
best_config = None
best_model_path = "best_model.pt"

trial_id = 0


# ------------------------------------------------------
# 5) Sweep
# ------------------------------------------------------
for (thresh,
     use_past_y,
     use_graph,
     use_comments) in itertools.product(
        thresholds,
        use_past_y_opts,
        use_graph_opts,
        use_comments_opts
    ):

    if not use_comments and not use_past_y:
        continue

    trial_id += 1

    print(
        f"\n=== Trial {trial_id} ===\n"
        f"threshold={thresh}, "
        f"use_past_y={use_past_y}, "
        f"use_graph={use_graph}, "
        f"use_comments={use_comments}"
    )

    model = TemporalTruthModel(
        COMMENT_DIM,
        use_past_y=use_past_y,
        use_graph=use_graph,
        use_comments=use_comments
    ).to(DEVICE)

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=LR,
        weight_decay=1e-4
    )

    # NO pos_weight (soft labels!)
    criterion = FocalLoss(gamma=2.0)

    user_ids = list(temporal_graph.user_nodes.keys())

    best_trial_macro = -1
    patience_ctr = 0

    # ---------------- TRAIN ----------------
    for epoch in range(EPOCHS):

        model.train()

        user_states = {
            u: torch.zeros(USER_HIDDEN, device=DEVICE)
            for u in user_ids
        }

        total_loss = 0.0

        for t in train_ts:

            optimizer.zero_grad()

            logits, labels, user_states = model.forward_one_step(
                temporal_graph, t, user_states
            )

            loss = criterion(logits, labels)
            loss.backward()

            # GRADIENT CLIPPING
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            optimizer.step()

            total_loss += loss.item()

            # Detach hidden states (prevent explosion)
            for u in user_states:
                user_states[u] = user_states[u].detach()

        # ---------------- VALIDATION ----------------
        model.eval()
        with torch.no_grad():

            user_states = {
                u: torch.zeros(USER_HIDDEN, device=DEVICE)
                for u in user_ids
            }

            val_logits_all = []
            val_labels_all = []

            for t in val_ts:
                logits, labels, user_states = model.forward_one_step(
                    temporal_graph, t, user_states
                )

                val_logits_all.append(logits)
                val_labels_all.append(labels)

            val_logits = torch.cat(val_logits_all)
            val_labels = torch.cat(val_labels_all)

            metrics = compute_metrics(val_logits, val_labels, thresh)
            macro_f1 = metrics["macro_f1"]

        print(
            f"Epoch {epoch:02d} | "
            f"Loss: {total_loss:.3f} | "
            f"Val Acc: {metrics['accuracy']:.3f} | "
            f"F1 Class 1: {metrics['f1_class_1']:.3f} | "
            f"Precision Class 1: {metrics['precision_class_1']:.3f} | "
            f"Recall Class 1: {metrics['recall_class_1']:.3f} | "

            f"F1 Class 0: {metrics['f1_class_0']:.3f} | "
            f"Precision Class 0: {metrics['precision_class_0']:.3f} | "
            f"Recall Class 0: {metrics['recall_class_0']:.3f} | "
            f"Macro F1: {macro_f1:.3f}"
        )

        if macro_f1 > best_trial_macro + 1e-4:
            best_trial_macro = macro_f1
            patience_ctr = 0
            torch.save(model.cpu().state_dict(), "temp_trial_best.pt")
            model.to(DEVICE)
        else:
            patience_ctr += 1
            if patience_ctr >= PATIENCE:
                print("Early stopping.")
                break

    results.append({
        "threshold": thresh,
        "use_past_y": use_past_y,
        "use_graph": use_graph,
        "use_comments": use_comments,
        "best_macro_f1": best_trial_macro
    })

    if best_trial_macro > best_macro_f1:
        best_macro_f1 = best_trial_macro
        best_config = results[-1]
        torch.save(model.cpu().state_dict(), best_model_path)
        model.to(DEVICE)

    del model
    del optimizer
    torch.cuda.empty_cache()
    gc.collect()


# ------------------------------------------------------
# 6) Save results
# ------------------------------------------------------
results_df = pd.DataFrame(results).sort_values(
    "best_macro_f1", ascending=False
)

results_df.to_csv("full_sweep_results.csv", index=False)

print("\n===== SWEEP COMPLETE =====")
print("Best configuration:")
print(best_config)
print("\nResults saved to full_sweep_results.csv")
print("Best model saved to best_model.pt")


Using device: cuda
COMMENT_DIM = 129
Train: 0 → 20
Val  : 27

=== Trial 1 ===
threshold=0.3, use_past_y=False, use_graph=True, use_comments=True


In [ ]:
# ================================
# TEMPORAL TRAIN / TEST + F1 SCORE
# ================================

import torch

# -------------------------------------------------------
# 1) Determine comment feature dimensionality
# -------------------------------------------------------
any_user = next(
    u for u in temporal_graph.user_nodes.values()
    if len(u.comments) > 0
)
any_comment = next(iter(any_user.comments.values()))
COMMENT_DIM = any_comment.feature_vec.shape[0]
print("COMMENT_DIM =", COMMENT_DIM)


# -------------------------------------------------------
# 2) Device
# -------------------------------------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", DEVICE)


# -------------------------------------------------------
# 3) Temporal split (by time interval)
# -------------------------------------------------------
T = len(temporal_graph.time_intervall.time_intervalls)

TRAIN_RATIO = 0.8
T_TRAIN = int(T * TRAIN_RATIO)

train_timesteps = list(range(0, T_TRAIN))
test_timesteps  = list(range(T_TRAIN, T))

print(f"Train timesteps: 0 → {T_TRAIN-1}")
print(f"Test timesteps:  {T_TRAIN} → {T-1}")


# -------------------------------------------------------
# 4) Initialize model
# -------------------------------------------------------
print("Initializing model with use_past_y = True")
model = TemporalTruthModel(
    COMMENT_DIM,
    use_past_y=True
).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = torch.nn.BCEWithLogitsLoss()


# -------------------------------------------------------
# 5) Helper: accuracy + F1
# -------------------------------------------------------
def compute_metrics(logits, labels, threshold=0.5):
    probs = torch.sigmoid(logits)
    preds = (probs > threshold).float()

    tp = ((preds == 1) & (labels == 1)).sum().item()
    fp = ((preds == 1) & (labels == 0)).sum().item()
    fn = ((preds == 0) & (labels == 1)).sum().item()

    precision = tp / (tp + fp + 1e-8)
    recall    = tp / (tp + fn + 1e-8)
    f1        = 2 * precision * recall / (precision + recall + 1e-8)

    acc = (preds == labels).float().mean().item()

    return acc, precision, recall, f1


# -------------------------------------------------------
# 6) Training loop (TRAIN ONLY on early timesteps)
# -------------------------------------------------------
EPOCHS = 20
print("\nStarting training for", EPOCHS, "epochs...\n")

for epoch in range(EPOCHS):
    model.train()
    optimizer.zero_grad()

    logits_all, labels_all = model(temporal_graph)

    # ---- TRAIN LOSS (only train timesteps) ----
    train_loss = sum(
        criterion(logits_all[t], labels_all[t])
        for t in train_timesteps
    )

    train_loss.backward()
    optimizer.step()

    # ---- TRAIN METRICS ----
    with torch.no_grad():
        train_logits = torch.cat([logits_all[t] for t in train_timesteps])
        train_labels = torch.cat([labels_all[t] for t in train_timesteps])
        acc, p, r, f1 = compute_metrics(train_logits, train_labels)

    print(
        f"Epoch {epoch:02d} | "
        f"Train loss: {train_loss.item():.4f} | "
        f"Acc: {acc:.3f} | "
        f"F1: {f1:.3f}"
    )


# -------------------------------------------------------
# 7) Evaluation on TEST timesteps
# -------------------------------------------------------
model.eval()

with torch.no_grad():
    logits_all, labels_all = model(temporal_graph)

    test_logits = torch.cat([logits_all[t] for t in test_timesteps])
    test_labels = torch.cat([labels_all[t] for t in test_timesteps])

    acc, p, r, f1 = compute_metrics(test_logits, test_labels)

print("\n===== TEST RESULTS =====")
print(f"Accuracy : {acc:.3f}")
print(f"Precision: {p:.3f}")
print(f"Recall   : {r:.3f}")
print(f"F1 score : {f1:.3f}")
print("========================\n")


# -------------------------------------------------------
# 8) Save model
# -------------------------------------------------------
torch.save(model.state_dict(), "temporal_truth_model.pt")
print("Model saved to temporal_truth_model.pt")



In [ ]:
len(graph.comment_df)

In [ ]:
graph = temporal_graph.TemporalGraph()
graph.clean_nodes(True, True, False)


In [ ]:
import random
dangerous_users=[6609,211,4273,8242,1772,6559,9365,3321,4248,8759,455,9264,4869,4210,8760,5807,805,8751,8256,6157]
for dangerous_user in dangerous_users:
    print(graph.nodes_dict[str(dangerous_user)].username)
# value = random.choice(list(graph.nodes_dict.values()))

value = graph.nodes_dict["6609"]
value.node_info()


In [ ]:
graph.find_truth("807621")

In [ ]:
for truth_id in graph.nodes_dict["20054"].truths:
    print(type(truth_id))


In [ ]:
graph.create_pyvis_representation()


In [ ]:
following_edges_per_node = []
follower_edges_per_node = []
messages_per_node = []

counter_follower = 0
counter_following = 0
counter_message = 0

zero_counter_following = 0
zero_counter_follower = 0
zero_counter_message = 0

for key in graph.nodes_dict.keys():
    if len(graph.nodes_dict[key].following_edges) != 0:
        following_edges_per_node.append(len(graph.nodes_dict[key].following_edges))
        counter_following += 1
    else:
        zero_counter_following += 1

    if len(graph.nodes_dict[key].truths) != 0:
        messages_per_node.append(len(graph.nodes_dict[key].truths))
        counter_message += 1
    else:
        zero_counter_message += 1

    if len(graph.nodes_dict[key].follower_edges) != 0:
        follower_edges_per_node.append(len(graph.nodes_dict[key].follower_edges))
        counter_follower += 1
    else:
        zero_counter_follower += 1

print(f"nodes with no known followings: {zero_counter_following} nodes with followings:{counter_following}")
print(f"nodes with no known messages: {zero_counter_message} with known messages:{counter_message}")
print(f"nodes with no known followers: {zero_counter_follower} nodes with followers:{counter_follower}")


In [ ]:
import numpy as np

print(np.median(following_edges_per_node))
plt.hist(following_edges_per_node, bins=3000)
plt.xscale("log")
plt.yscale("log")


In [ ]:
plt.hist(follower_edges_per_node, bins=3000)
plt.xscale("log")
plt.yscale("log")


In [ ]:
plt.hist(messages_per_node, bins=3000)
plt.xscale("log")
plt.yscale("log")
